# Sprint 2 - Lab 2 - Entrenamiento 1

Este proyecto tiene como objetivo construir y entrenar una red neuronal feed-forward utilizando PyTorch para predecir las puntuaciones de los estudiantes en función de las horas de estudio, utilizando el 'Students Score Dataset'. Los participantes aplicarán técnicas de preprocesamiento de datos, entrenarán una red neuronal, y evaluarán el rendimiento del modelo utilizando métricas de regresión como R2, MAE, y RMSE.


## Preparación y preprocesamiento de los datos

El dataset student_scores.csv se carga correctamente. El código divide el dataset en un conjunto de entrenamiento y un conjunto de prueba (75%-25%) utilizando train_test_split de Scikit-learn, lo cual cumple con el requisito de preparar y particionar los datos.

In [41]:
# Importar librerías necesarias
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt


### 1. Cargar y explorar dataset

In [42]:
df = pd.read_csv('student_scores.csv')
df.sample(10)


,Hours,Scores
2,3.2,27
22,3.8,35
3,8.5,75
6,9.2,88
8,8.3,81
9,2.7,25
23,6.9,76
20,2.7,30
21,4.8,54
4,3.5,30


In [43]:
df.shape

(25, 2)

In [44]:
df.describe()

,Hours,Scores
count,25.000000,25.000000
mean,5.012000,51.480000
std,2.525094,25.286887
min,1.100000,17.000000
25%,2.700000,30.000000
50%,4.800000,47.000000
75%,7.400000,75.000000
max,9.200000,95.000000


### 2. Preparar dataset para la regresión 

In [45]:
X = df[['Hours']].values 
Y = df[['Scores']].values 

#Dividir en datos de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(
    X,Y,test_size = 0.25, random_state=42
)

#Verificar tamaño subconjunto
print("Tamaño de entrenamiento:", X_train.shape, y_train.shape)
print("Tamaño de prueba:", X_test.shape, y_test.shape)

Tamaño de entrenamiento: (18, 1) (18, 1)
Tamaño de prueba: (7, 1) (7, 1)


### 3. Construcción de la red Neuronal 

Convertir vectores a tensores

In [46]:
X_train_t = torch.tensor(X_train,dtype = torch.float32)
X_test_t = torch.tensor(X_test, dtype =torch.float32)
y_train_t = torch.tensor(y_train,dtype = torch.float32)
y_test_t = torch.tensor(y_test, dtype =torch.float32)

Crear Red

In [47]:
class StudentScoreNN(nn.Module):
    def __init__(self):
        super(StudentScoreNN, self).__init__()

        self.fc1 = nn.Linear(1,64)

        self.fc2 = nn.Linear(64,32)

        self.out = nn.Linear(32,1)

        self.relu = nn.ReLU()

    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.out(x)
        return x

#instancia del modelo

model = StudentScoreNN()

# Verificar que tiene parámetros
print(sum(p.numel() for p in model.parameters()), "parámetros entrenables")


2241 parámetros entrenables


### 4. Entrenamiento de la red neuronal

In [32]:
criterion = nn.MSELoss() #Definir función de pérdida
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [48]:
epochs = 200
losses = []

for epoch in range(epochs):

    y_pred = model(X_train_t)
    loss = criterion(y_pred,y_train_t)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    losses.append(loss.item)

    # Mostrar progreso cada 20 épocas
    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {loss.item():.4f}")


Epoch [20/200] - Loss: 3223.4783
Epoch [40/200] - Loss: 3223.4783
Epoch [60/200] - Loss: 3223.4783
Epoch [80/200] - Loss: 3223.4783
Epoch [100/200] - Loss: 3223.4783
Epoch [120/200] - Loss: 3223.4783
Epoch [140/200] - Loss: 3223.4783
Epoch [160/200] - Loss: 3223.4783
Epoch [180/200] - Loss: 3223.4783
Epoch [200/200] - Loss: 3223.4783


In [53]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Desactivar gradientes (solo evaluación)
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_t)

# Convertir tensores a arrays de NumPy
y_pred_np = y_pred_test.numpy()
y_true_np = y_test_t.numpy()

# Calcular métricas
r2 = r2_score(y_true_np, y_pred_np)
mae = mean_absolute_error(y_true_np, y_pred_np)
rmse = np.sqrt(mean_squared_error(y_true_np, y_pred_np))

print(f"🔹 R² Score: {r2:.4f}")
print(f"🔹 MAE:      {mae:.4f}")
print(f"🔹 RMSE:     {rmse:.4f}")


🔹 R² Score: -4.0856
🔹 MAE:      46.6948
🔹 RMSE:     51.7549
